In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np

In [3]:
# Define activations separately for indexing and instantiation
activation_names = ['sigmoid', 'tanh', 'relu', 'elu', 'selu', 'swish']
activation_map = {
    'sigmoid': nn.Sigmoid,
    'tanh': nn.Tanh,
    'relu': nn.ReLU,
    'elu': nn.ELU,
    'selu': nn.SELU,
    'swish': lambda: nn.SiLU(),  # swish ≈ SiLU in PyTorch
}

def decode_action(action_id):
    if 0 <= action_id <= 41:  # Dense Layer with some number of units
        units_list = [8, 16, 32, 64, 128, 256, 512]
        units_idx = action_id // len(activation_names)
        act_idx = action_id % len(activation_names)
        units = units_list[units_idx]
        activation_name = activation_names[act_idx]
        activation_fn = activation_map[activation_name]
        return ('dense', units, activation_name)  

    elif 42 <= action_id <= 44:  # Dropout layer
        dropout_rates = [0.0, 0.2, 0.5]
        return ('dropout', dropout_rates[action_id - 42])

    elif action_id == 45:  # BatchNorm layer
        return ('batchnorm',)

    elif action_id == 46:  # Stop building layers
        return ('stop',)

    else:
        raise ValueError(f"Invalid action id: {action_id}")


In [5]:
class NASMLPEnv(Env):
    def __init__(self, dataset, max_layers=10):
        self.dataset = dataset  
        self.max_layers = max_layers
        self.action_space = Discrete(47)
        self.observation_space = Box(low=0, high=1, shape=((max_layers + 10) * 3,), dtype=np.float32)  # 3 inputs per layer, layer type, number of units, and activation function
        self.architecture = []
        self.done = False

        self.best_architecture = None
        self.best_reward = -float('inf')
        self.architecture_log = []

        self.max_parameters = self._estimate_max_parameters()

    def reset(self):
        if self.architecture:
            self.architecture_log.append({
                'architecture': list(self.architecture),
                'reward': 0,
                'accuracy': None,
                'complexity': None,
                'completed': False,
                'termination_reason': 'incomplete'
            })
        self.architecture = []
        self.done = False
        return self._get_obs()

    def _estimate_max_parameters(self):
        input_dim = self.dataset[0].shape[1]  
        max_units = 512
        total_params = 0
    
        for _ in range(self.max_layers):
            total_params += input_dim * max_units + max_units  
            input_dim = max_units
    
        output_dim = self.dataset[1].shape[1]
        total_params += max_units * output_dim + output_dim
    
        return total_params


    def step(self, action_id):
        decoded = decode_action(action_id)

        dense_count = sum(1 for layer in self.architecture if layer[0] == 'dense')
        if decoded[0] == 'stop' or (decoded[0] == 'dense' and dense_count >= self.max_layers):
            self.done = True
            reward, acc, complexity = self._evaluate_model()  

            # Log architecture, reward, accuracy, and complexity
            self.architecture_log.append({
                'architecture': list(self.architecture),
                'reward': reward,
                'accuracy': acc,
                'complexity': complexity,
                'completed': True,
                'termination_reason': 'stop' if decoded[0] == 'stop' else 'max_dense'
            })
        
    
            print(f"\n🎯 Final Architecture: {self.architecture}")
            print(f"🏆 Validation Accuracy (acc): {acc:.2f}%")
            print(f"⚙️  Complexity (number of layers): {complexity}")
            print(f"🏅 Reward (acc - penalty): {reward:.2f}%\n")

            # Track best
            if reward > self.best_reward:
                self.best_reward = reward
                self.best_architecture = list(self.architecture)
                self.best_accuracy = acc  
                self.best_complexity = complexity 
                print(f"🌟 New Best Architecture Found with Reward: {reward:.2f}%")
        else:
            self.architecture.append(decoded)
            reward = 0

        return self._get_obs(), reward, self.done, {}

    def _get_obs(self):
        obs = np.zeros(self.observation_space.shape[0])
        for i, layer in enumerate(self.architecture):
            base = i * 3
            if layer[0] == 'dense':
                obs[base] = layer[1] / 512  # normalized units
                obs[base + 1] = activation_names.index(layer[2]) / (len(activation_names) - 1)
                obs[base + 2] = 0
            elif layer[0] == 'dropout':
                obs[base] = layer[1]
                obs[base + 1] = -1
                obs[base + 2] = 1
            elif layer[0] == 'batchnorm':
                obs[base] = -1
                obs[base + 1] = -1
                obs[base + 2] = 2
        return obs

    def _evaluate_model(self):
        X_train, y_train, X_val, y_val = self.dataset

        model = nn.Sequential()
        input_dim = X_train.shape[1]

        for i, layer in enumerate(self.architecture):
            if layer[0] == 'dense':
                model.add_module(f"fc{i}", nn.Linear(input_dim, layer[1]))
                model.add_module(f"act{i}", activation_map[layer[2]]())
                input_dim = layer[1]
            elif layer[0] == 'dropout':
                model.add_module(f"dropout{i}", nn.Dropout(p=layer[1]))
            elif layer[0] == 'batchnorm':
                model.add_module(f"bn{i}", nn.BatchNorm1d(input_dim))

        model.add_module("output", nn.Linear(input_dim, y_train.shape[1]))

        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=0.01) # can make the RL choose optimizer and LR also here
        train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)

        model.train()
        for _ in range(10):  
            for xb, yb in train_loader:
                optimizer.zero_grad()
                output = model(xb)
                loss = loss_fn(output, yb)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            preds = model(X_val)
            acc = (preds.argmax(dim=1) == y_val.argmax(dim=1)).float().mean().item()

        no_of_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        max_params = self._estimate_max_parameters()
        alpha = 0.8 

        reward = (alpha * acc * 100) - ((1 - alpha) * (no_of_params*100/max_params))

        return reward, acc * 100, (no_of_params*100/max_params)

In [7]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from torch.nn.functional import one_hot

import openml
dataset = openml.datasets.get_dataset(23512)
X, y, _, _ = dataset.get_data(target=dataset.default_target_attribute)

df = pd.concat([X, y], axis=1)

df.dropna(inplace=True)

X = df.drop(columns=[dataset.default_target_attribute])
y = df[dataset.default_target_attribute]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

le = LabelEncoder()
y_encoded = le.fit_transform(y)  

y_oh = one_hot(torch.tensor(y_encoded)).float()

X_train_np, X_val_np, y_train_tensor, y_val_tensor = train_test_split(
    X_scaled, y_oh, test_size=0.1, random_state=42)

X_train_tensor = torch.tensor(X_train_np).float()
X_val_tensor = torch.tensor(X_val_np).float()

In [8]:
dataset = (X_train_tensor, y_train_tensor, X_val_tensor, y_val_tensor)

In [9]:
print(X_train_tensor.shape)  # [n_train, n_features]
print(y_train_tensor.shape)  # [n_train, 2]  (one-hot binary)

torch.Size([88244, 28])
torch.Size([88244, 2])


In [10]:
# Pass into your NAS environment
env = NASMLPEnv(dataset=dataset, max_layers=10)

In [15]:
from stable_baselines3 import DQN

import time

start_time = time.time()

model = DQN(
    "MlpPolicy",
    env,
    gamma=0.99,                       
    exploration_initial_eps=1.0,     
    exploration_final_eps=0.05,      
    exploration_fraction=0.1,        
    verbose=1,
    tensorboard_log="./nas_logs/"
)

#model = DQN("MlpPolicy", env, verbose=1, tensorboard_log="./nas_logs/")
model.learn(total_timesteps=10000)

end_time = time.time()
elapsed_time = end_time - start_time

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/opt/anaconda3/lib/python3.12/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Logging to ./nas_logs/DQN_26

🎯 Final Architecture: [('dropout', 0.5), ('dense', 512, 'selu'), ('dense', 256, 'relu'), ('dense', 8, 'sigmoid'), ('dense', 64, 'sigmoid'), ('dense', 64, 'elu'), ('dense', 512, 'swish'), ('dense', 256, 'tanh'), ('dense', 32, 'relu'), ('dense', 512, 'selu'), ('dense', 32, 'sigmoid')]
🏆 Validation Accuracy (acc): 46.68%
⚙️  Complexity (number of layers): 15.092920432073916
🏅 Reward (acc - penalty): 34.33%

🌟 New Best Architecture Found with Reward: 34.33%

🎯 Final Architecture: [('dense', 256, 'tanh'), ('dropout', 0.2), ('dense', 64, 'sigmoid'), ('dense', 512, 'elu'), ('dense', 256, 'selu'), ('dense', 512, 'sigmoid'), ('dense', 8, 'selu'), ('dense', 32, 'swish'), ('dense', 8, 'elu'), ('dense', 32, 'tanh'), ('dense', 16, 'tanh')]
🏆 Validation Accuracy (acc): 53.32%
⚙️  Complexity (number of layers): 13.68068786248129
🏅 Reward (acc - penalty): 39.92%

🌟 New Best Architecture Found with Reward: 39.92%

🎯 Final Architecture: [('dense', 256, 'elu'), ('dense', 16,

In [16]:
model.save("nas_agent_checkpoint_alpha_0.8_23512")

In [17]:
# load previous model
# model = DQN.load("nas_agent_checkpoint")

In [26]:
# analyze the results

In [21]:
print(elapsed_time)

96248.45004296303


In [23]:
print(env.best_architecture)

[('dense', 512, 'sigmoid')]


In [25]:
# interesting result: look into it, pretty exciting

In [27]:
print(f"\n Best Reward: {env.best_reward:.2f}%")


 Best Reward: 57.55%


In [29]:
print(f"\n Best Accuracy: {env.best_accuracy:.2f}%")


 Best Accuracy: 72.11%


In [31]:
print(f"\n Best Complexity: {env.best_complexity:.2f}%")


 Best Complexity: 0.67%


In [33]:
import pandas as pd

flat_data = []

for arch_id, run in enumerate(env.architecture_log):
    reward = run['reward']
    accuracy = run.get('accuracy', None)
    complexity = run.get('complexity', None)
    completed = run.get('completed', False)
    termination_reason = run.get('termination_reason', 'unknown')

    for i, layer in enumerate(run['architecture']):
        layer_type = layer[0]
        layer_info = {
            'arch_id': arch_id,
            'layer_num': i + 1,
            'layer_type': layer_type,
            'units': None,
            'activation': None,
            'dropout_rate': None,
            'reward': reward,
            'accuracy': accuracy,
            'complexity': complexity,
            'completed': completed,
            'termination_reason': termination_reason
        }

        if layer_type == 'dense':
            layer_info['units'] = layer[1]
            layer_info['activation'] = layer[2]

        elif layer_type == 'dropout':
            layer_info['dropout_rate'] = layer[1]

        flat_data.append(layer_info)

# Create DataFrame and export to CSV
df = pd.DataFrame(flat_data)
df.to_csv("architecture_analysis_updated.csv", index=False)

In [37]:
# still needs to look at why there is only 6728 rows
df

,arch_id,layer_num,layer_type,units,activation,dropout_rate,reward,accuracy,complexity,completed,termination_reason
0,0,1,dropout,NaN,None,0.5,34.325628,46.680266,15.092920,True,max_dense
1,0,2,dense,512.0,selu,NaN,34.325628,46.680266,15.092920,True,max_dense
2,0,3,dense,256.0,relu,NaN,34.325628,46.680266,15.092920,True,max_dense
3,0,4,dense,8.0,sigmoid,NaN,34.325628,46.680266,15.092920,True,max_dense
4,0,5,dense,64.0,sigmoid,NaN,34.325628,46.680266,15.092920,True,max_dense
...,...,...,...,...,...,...,...,...,...,...,...
6723,7216,1,batchnorm,NaN,None,NaN,0.000000,NaN,NaN,False,incomplete
6724,7244,1,dense,8.0,swish,NaN,55.267660,69.087201,0.010505,True,stop
6725,7245,1,dense,8.0,swish,NaN,0.000000,NaN,NaN,False,incomplete
6726,7247,1,dense,64.0,tanh,NaN,55.057253,68.842429,0.083453,True,stop


In [41]:
df.groupby('activation')['accuracy'].mean()

activation
elu        58.187054
relu       59.291178
selu       56.725523
sigmoid    53.877449
swish      58.579698
tanh       55.326050
Name: accuracy, dtype: float64

In [43]:
df.groupby('units')['accuracy'].mean()

units
8.0      53.862298
16.0     57.987943
32.0     59.184169
64.0     57.417682
128.0    59.076210
256.0    53.799047
512.0    58.109830
Name: accuracy, dtype: float64

In [ ]:
# analyze effect of different units in each layer
# analyze effect of different activation functions
# analyze the effect of different layers